In [ ]:
import chargingmodel

### Run the model
Most important arguments:
- *dbName*: Name of the database to store to. Default: *chargingmodel/output/[scenario]_[strategy].db*
- *scenario*: Set the EVSE scenario. Defines the availability and power of charging options at each location.
- *strategy*: Uncontrolled, Opt_county, Opt_National (See Paper)

In [ ]:
uncontrolled = "chargingmodel/output/MyUncontrolled.db"
chargingmodel.run(dbName=uncontrolled, strategy='Uncontrolled', n_worker=3)

In [ ]:
# Takes ~ 3min
# Here setting n_worker != 1 means running agents batches in parallel. Can lead to errors due to less frequent residual load updates.
# Default batchsize = 25, Default n_worker = 1
optimal = "chargingmodel/output/MyOptimized.db"
chargingmodel.run(dbName=optimal, strategy='Opt_National', verbose=False)

Create aggregated loads for each region or all regions combined.
The result is stored in the database.

In [ ]:
# All regions together
chargingmodel.postprocessing.processTotalLoad(uncontrolled)
chargingmodel.postprocessing.processTotalLoad(optimal)

In [ ]:
# All regions individually
chargingmodel.postprocessing.processRegionalLoad(uncontrolled)
chargingmodel.postprocessing.processRegionalLoad(optimal)

### Get output load as time series

In [ ]:
df_uncontrolled = chargingmodel.postprocessing.getProcessed(uncontrolled, region="Total")
df_optimal = chargingmodel.postprocessing.getProcessed(optimal, region="Total")

### Example: Plot the result for one week

In [ ]:
from datetime import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Get the original load (not part of the package)
df_noEVLoad = None
regions = ["Region_1", "Region_2", "Region_3"]
for region in regions:
    fn = "chargingmodel/input/residual-load/" + region + ".csv"
    if df_noEVLoad is None:
        df_noEVLoad = pd.read_csv(fn, sep=";", parse_dates=["TimeStamp"], index_col="TimeStamp")
        df_noEVLoad.rename(columns={"ResidualLoad_MW": region}, inplace=True)
    else:
        df_tmp = pd.read_csv(fn, sep=";", parse_dates=["TimeStamp"], index_col="TimeStamp")
        df_noEVLoad[region] = df_tmp["ResidualLoad_MW"]
df_noEVLoad['Total_MW'] = df_noEVLoad.loc[:, regions].sum(axis=1)

In [ ]:
week = slice(dt(2030, 9, 16), dt(2030, 9, 22, 23, 45))
x = df_noEVLoad.loc[week, 'Total_MW'].index
noEV = df_noEVLoad.loc[week, 'Total_MW'].values
load_uncontrolled = noEV + df_uncontrolled.loc[week, 'PowerMW']
load_optimal = noEV + df_optimal.loc[week, 'PowerMW']

In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
ax.plot(x, noEV, label="No EV")
ax.plot(x, load_uncontrolled, label="Uncontrolled")
ax.plot(x, load_optimal, label="Optimal")
_ = ax.set_ylabel("Load [MW]")
_ = ax.legend()